In [ ]:
import json
import os
import csv
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
from torch.cuda.amp import autocast, GradScaler
from torchvision.models import vgg16
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import cv2

# Transformaciones de datos
data_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])


In [ ]:
def load_annotations(file_path):
    """
    Carga las anotaciones preprocesadas desde un archivo JSON.
    """
    with open(file_path, 'r') as f:
        return json.load(f)

In [ ]:
class SkeletonDataset(Dataset):
    def __init__(self, annotations, image_dir, transform=None):
        """
        Inicializa el dataset para análisis de poses esqueléticas.
        """
        self.annotations = annotations
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        """
        Devuelve el tamaño del dataset.
        """
        return len(self.annotations)

    def __getitem__(self, idx):
        """
        Devuelve una imagen y su correspondiente vector de poses concatenadas.
        """
        image_name = self.annotations[idx]['Image'] + '.jpg'
        image_path = os.path.join(self.image_dir, image_name)
        image = Image.open(image_path)

        pose1 = self.annotations[idx]['Pose1']
        pose2 = self.annotations[idx]['Pose2']

        # Quitar fondo de la imagen
        image = remove_background(image)

        if self.transform:
            image = self.transform(image)

        pose = pose1 + pose2
        pose = torch.tensor(pose).view(-1)  # Vectorizar las poses
        return image, pose

In [ ]:
def remove_background(image):
    """
    Quita el fondo de una imagen utilizando técnicas de segmentación de contornos con OpenCV.
    """
    # Convertir imagen PIL a array de NumPy
    image_np = np.array(image)

    # Convertir a escala de grises
    gray = cv2.cvtColor(image_np, cv2.COLOR_RGB2GRAY)

    # Aplicar un umbral para segmentar la imagen
    _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY)

    # Encontrar contornos
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Crear una máscara vacía
    mask = np.zeros_like(gray)

    # Dibujar el contorno más grande en la máscara
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        cv2.drawContours(mask, [largest_contour], -1, (255), thickness=cv2.FILLED)

    # Aplicar la máscara a la imagen original
    result = cv2.bitwise_and(image_np, image_np, mask=mask)

    # Convertir de nuevo a imagen PIL
    return Image.fromarray(result)

In [ ]:
class VGG16ForPose(nn.Module):
    def __init__(self, num_keypoints=102):
        """
        Modifica VGG16 para predecir directamente puntos clave de poses (102 valores).
        """
        super(VGG16ForPose, self).__init__()
        self.backbone = vgg16(pretrained=True).features
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, num_keypoints)
        )

    def forward(self, x):
        """
        Pasada hacia adelante del modelo.
        """
        x = self.backbone(x)
        x = self.pool(x)
        x = self.fc(x)
        return x

In [ ]:
def train_one_epoch(model, train_loader, criterion, optimizer, scaler, device):
    """
    Entrena el modelo durante una época.
    """
    model.train()
    running_loss = 0.0

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()

        # AMP para precisión mixta
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, targets)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item() * inputs.size(0)

    return running_loss / len(train_loader.dataset)


In [ ]:
def validate_one_epoch(model, val_loader, criterion, scaler, device):
    """
    Valida el modelo durante una época.
    """
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item() * inputs.size(0)

    return val_loss / len(val_loader.dataset)

In [ ]:
def train_and_validate(train_loader, val_loader, model, criterion, optimizer, scaler, device, num_epochs=25, patience=5):
    """
    Entrena y valida el modelo, implementando early stopping. Calcula métricas adicionales.
    """
    if not torch.cuda.is_available():
        print("CUDA no está disponible. Asegúrate de que tu GPU esté configurada correctamente.")
    else:
        print(f"Usando GPU: {torch.cuda.get_device_name(torch.cuda.current_device())}")

    best_val_loss = float('inf')
    early_stopping_counter = 0
    epoch_results = []  # Para almacenar los resultados de cada época

    for epoch in range(num_epochs):
        # Entrenamiento
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, scaler, device)
        print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}')

        # Validación
        val_loss = validate_one_epoch(model, val_loader, criterion, scaler, device)
        print(f'Epoch {epoch+1}, Validation Loss: {val_loss:.4f}')

        # Calcular métricas adicionales en el conjunto de validación
        y_true, y_pred = [], []
        model.eval()
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                with autocast():
                    outputs = model(inputs)
                y_true.extend(targets.cpu().numpy())
                y_pred.extend(outputs.cpu().numpy())

        y_true = np.array(y_true).reshape(-1, 102)  # Convertir a formato batch_size x 102
        y_pred = np.array(y_pred).reshape(-1, 102)

        # Calcular métricas
        mae = mean_absolute_error(y_true, y_pred)
        mse = mean_squared_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)
        std_ae = np.mean(np.abs(y_true - y_pred), axis=1).std()

        print(f'Metrics -> MAE: {mae:.4f}, MSE: {mse:.4f}, R²: {r2:.4f}, STD AE: {std_ae:.4f}')

        # Guardar resultados de la época
        epoch_results.append({
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'mae': mae,
            'mse': mse,
            'r2': r2,
            'std_ae': std_ae
        })

        # Early Stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            early_stopping_counter = 0
            torch.save(model.state_dict(), 'vgg16_best_model.pth')
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= patience:
                print("Early stopping triggered")
                break

        torch.cuda.empty_cache()

    # Guardar resultados en un archivo CSV
    with open('epoch_results.csv', 'a', newline='') as csvfile:
        fieldnames = ['epoch', 'train_loss', 'val_loss', 'mae', 'mse', 'r2', 'std_ae']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(epoch_results)

In [ ]:
def cross_validation_kfold(dataset, k_folds=5, batch_size=8, num_epochs=25, patience=5):
    """
    Realiza validación cruzada con K-Fold en el dataset y utiliza múltiples GPUs.
    """
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    if torch.cuda.device_count() > 1:
        print(f"Usando {torch.cuda.device_count()} GPUs")

    num_workers = 4

    for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
        print(f'Fold {fold+1}/{k_folds}')

        train_subset = torch.utils.data.Subset(dataset, train_idx)
        val_subset = torch.utils.data.Subset(dataset, val_idx)

        train_loader = DataLoader(
            train_subset,
            batch_size=batch_size,
            shuffle=True,
            num_workers=num_workers,
            pin_memory=True
        )
        val_loader = DataLoader(
            val_subset,
            batch_size=batch_size,
            num_workers=num_workers,
            
            pin_memory=True
        )

        model = VGG16ForPose(num_keypoints=102).to(device)

        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)

        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        scaler = GradScaler()

        train_and_validate(train_loader, val_loader, model, criterion, optimizer, scaler, device, num_epochs, patience)


In [ ]:
def main():
    """
    Función principal para cargar datos y ejecutar validación cruzada.
    """
    annotations_path = '../mnt/V3/annotations/annotations_preprocessed.json'
    image_dir = '../mnt/V3/images'

    annotations = load_annotations(annotations_path)

    dataset = SkeletonDataset(annotations, image_dir=image_dir, transform=data_transforms)

    cross_validation_kfold(dataset, k_folds=5, batch_size=8, num_epochs=15, patience=3)

if __name__ == '__main__':
    main()


Fold 1/5


c:\Users\Tuf Gaming\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Tuf Gaming\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/25, Loss: 0.01611253821070558
Validation Loss: 0.013531259782695275
Epoch 2/25, Loss: 0.009558142768751306
Validation Loss: 0.008524348457556808
Epoch 3/25, Loss: 0.007512813944025657
Validation Loss: 0.0070005322940907685
Epoch 4/25, Loss: 0.006225689877728949
Validation Loss: 0.005846129299876266
Epoch 5/25, Loss: 0.005409117011333794
Validation Loss: 0.005065812594349092
Epoch 6/25, Loss: 0.0047144331753104
Validation Loss: 0.00455638022444485
Epoch 7/25, Loss: 0.004206075403196998
Validation Loss: 0.004315639101407997
Epoch 8/25, Loss: 0.003804756429000767
Validation Loss: 0.00408530786693083
Epoch 9/25, Loss: 0.003462969511048952
Validation Loss: 0.0037049341363154954
Epoch 10/25, Loss: 0.0031801171279985524
Validation Loss: 0.003557995717702434
Epoch 11/25, Loss: 0.0029444977487532126
Validation Loss: 0.003343730706971499
Epoch 12/25, Loss: 0.002724811281416246
Validation Loss: 0.0032343056724440386
Epoch 13/25, Loss: 0.0025306374819807575
Validation Loss: 0.0031088039059